# Preamble

In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
import seaborn as sns
import petpal.kinetic_modeling.graphical_analysis as pet_grph
import petpal.kinetic_modeling.parametric_images as pet_pim
import nibabel
import json
import os
from importlib import reload

sns.set()
sns.set_style("ticks")
sns.set_context("paper", font_scale=1.6)
plt.rcParams['font.family'] ='StixGeneral'
plt.rcParams['mathtext.fontset'] = 'stix'

/Users/furqandar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Using Noise Free TACs

## Loading In TAC Parameters

In [7]:
def count_parameter_sets(model_directories: dict) -> dict:
    model_nums = {}
    for model, directory in model_directories.items():
        model_nums[model] = len([f for f in os.listdir(directory) if os.path.splitext(f)[1] == '.json'])
    return model_nums

In [8]:
DIR_PRE = "../../"
model_dirs = {"1tcm":f"{DIR_PRE}data/tcm_tacs/1tcm/",
              "serial_2tcm_k4zero":f"{DIR_PRE}data/tcm_tacs/serial_2tcm_k4zero/",
              "serial_2tcm":f"{DIR_PRE}data/tcm_tacs/serial_2tcm/"}

In [9]:
MODELS_NUMS = count_parameter_sets(model_dirs)
MODELS_NUMS

In [11]:
def safe_load_json(file_path):
    try:
        with open(file_path) as file:
            return json.load(file)
    except Exception as e:
        print(f"Couldn't read file {file_path}. Error: {e}")
        return None

In [12]:
model_params = {
    model: {f'set-{i:02}': safe_load_json(f'{DIR_PRE}data/tcm_tacs/{model}/params_{model}_set-{i:02}.json') for i in
            range(num)} for model, num in MODELS_NUMS.items()}

## Loading In TACs

In [195]:
def safe_load_tac(filename, stride=5):
    try:
        return np.array(np.loadtxt(filename)[::stride].T, dtype=float, order='C')
    except Exception as e:
        print(f"Couldn't read file {filename}. Error: {e}")
        return None

In [196]:
pTAC = safe_load_tac('../../data/tcm_tacs/fdg_plasma_clamp_evenly_resampled.txt')

In [197]:
all_tacs = {}
all_tacs['noise-free'] = {a_model: {f'set-{i:02}': safe_load_tac(
    f'{DIR_PRE}data/tcm_tacs/{a_model}/noise_free/tac_{("_".join(a_model.split("_")[1:]) if "serial_2tcm" in a_model else a_model)}_set-{i:02}.txt')
    for i in range(num)} for a_model, num in MODELS_NUMS.items()}
all_tacs['gaussian-noise'] = {a_model: {f'set-{i:02}': safe_load_tac(
    f'{DIR_PRE}data/tcm_tacs/{a_model}/gaussian_noise/tac_{("_".join(a_model.split("_")[1:]) if "serial_2tcm" in a_model else a_model)}_set-{i:02}.txt')
    for i in range(num)} for a_model, num in MODELS_NUMS.items()}

## Patlak Plots

In [41]:
from cycler import cycler

In [132]:
dar_cycler = cycler(color=['red', 'blue', 'green', 'yellow', 'purple', 'cyan', 'magenta'])
plt.rc('axes', prop_cycle=dar_cycler)

In [19]:
def gen_patlak_plot(pTAC: np.ndarray[float, float], tTAC: np.ndarray[float, float]):
    x = pet_grph.cumulative_trapezoidal_integral(xdata=pTAC[0], ydata=pTAC[1]) / pTAC[1]
    y = tTAC[1]/pTAC[1]
    return np.array([x, y])

In [21]:
patlak_xLab = r"$\frac{\int_{0}^{t}C_\mathrm{P}(s)\mathrm{d}s}{C_\mathrm{P}(t)}$"
patlak_yLab = r"$\frac{R(t)}{C_\mathrm{P}(t)}$"

In [22]:
def params_dict_to_label(params):
    tmp = []
    for p, val in params.items():
        tmp.append(f"$k_{{{p[1:]}}}={float(val):<.2f}$")
    return "; ".join(tmp)
params_dict_to_label(model_params['1tcm']['set-00']['params'])

In [255]:
patlak_plot_kwargs = {'lw':1, 'alpha':0.9, 'ms':8, 'marker':'.'}

def calc_patlak_x_and_y(pTAC, tTAC):
    good_points = np.argwhere(pTAC[1] != 0.0).T[0]
    xAr = pet_grph.cumulative_trapezoidal_integral(xdata=pTAC[0], ydata=pTAC[1])[good_points] / pTAC[1][good_points]
    yAr = tTAC[1][good_points] / pTAC[1][good_points]
    return xAr, yAr
    
def add_linear_patlak_plot(pTAC, tTAC, axObj = None):
    if axObj is None:
        axObj = plt.gca()
    
    xAr, yAr = calc_patlak_x_and_y(pTAC=pTAC, tTAC=tTAC)
    
    axObj.plot(xAr, yAr, '--', **patlak_plot_kwargs, zorder=1, color='black')
    
    return None

def add_patlak_plot_shading(pTAC, tTAC, t_thresh_in_mins, axObj = None):
    if axObj is None:
        axObj = plt.gca()
    good_points = np.argwhere(pTAC[1] != 0.0).T[0]
    t_thresh = pet_grph.get_index_from_threshold(times_in_minutes=pTAC[0][good_points], 
                                                 t_thresh_in_minutes=t_thresh_in_mins)
    
    xAr, yAr = calc_patlak_x_and_y(pTAC=pTAC, tTAC=tTAC)
    
    axObj.axvspan(xAr[t_thresh], xAr[-1], color='gray', alpha=0.2, zorder=0)
    return None

def add_patlak_fit_points(pTAC, tTAC, t_thresh_in_mins, axObj = None):
    if axObj is None:
        axObj = plt.gca()
    good_points = np.argwhere(pTAC[1] != 0.0).T[0]
    t_thresh = pet_grph.get_index_from_threshold(times_in_minutes=pTAC[0][good_points], 
                                                 t_thresh_in_minutes=t_thresh_in_mins)
    
    xAr, yAr = calc_patlak_x_and_y(pTAC=pTAC, tTAC=tTAC)
    
    axObj.plot(xAr[t_thresh:], yAr[t_thresh:], 'o-', alpha=0.9, ms='5', zorder=2, color='blue')
    return None

def add_patlak_line(pTAC, tTAC, t_thresh_in_mins, axObj = None):
    if axObj is None:
        axObj = plt.gca()
    
    slope, intercept, rSq = pet_grph.patlak_analysis_with_rsquared(input_tac_values=pTAC[1],
                                                             region_tac_values=tTAC[1],
                                                             tac_times_in_minutes=pTAC[0],
                                                             t_thresh_in_minutes=t_thresh_in_mins)
    xAr, _ = calc_patlak_x_and_y(pTAC=pTAC, tTAC=tTAC)
    yAr = slope*xAr + intercept
    
    axObj.plot(xAr, yAr, '-', color='orange', lw=2.5, zorder=3, label=f"$K_1=${slope:<5.3f}\n$V_T=${intercept:<5.3f}\n$R^2=${rSq:<5.3f}")
    
    
def gen_patlak_figure(pTAC, tTAC, t_thresh_in_mins, figObj=None):
    
    if figObj is None:
        fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=[8, 4], 
                               dpi=100, linewidth=3.0, edgecolor='k')
        ax = ax.flatten()
    print(type(fig))
    for axId, anAx in enumerate(ax.flatten()):
        add_linear_patlak_plot(pTAC=pTAC, tTAC=tTAC, axObj=anAx)
        add_patlak_plot_shading(pTAC=pTAC, tTAC=tTAC, t_thresh_in_mins=40.0, axObj=anAx)
        add_patlak_line(pTAC=pTAC, tTAC=tTAC, t_thresh_in_mins=40.0, axObj=anAx)
        add_patlak_fit_points(pTAC=pTAC, tTAC=tTAC, t_thresh_in_mins=30.0, axObj=anAx)

    ax[0].set(title=f"Linear Plot")
    ax[1].set(title=f"Log-Log Plot", xscale='log', yscale='log')
    ax[0].set_xlabel(patlak_xLab, fontsize=20)
    ax[0].set_ylabel(patlak_yLab, fontsize=20)
    ax[1].set_xlabel(patlak_xLab, fontsize=20)
    ax[1].set_ylabel(patlak_yLab, fontsize=20)
    fig.suptitle(f"Patlak Plot", ha='center', va='center')
    fig.legend(*ax[0].get_legend_handles_labels(),
               bbox_to_anchor=(1.0, 0.85), loc='upper left', 
               title='Patlak Analysis', edgecolor='black')
    

### Example Plots

In [256]:
for a_model, num in MODELS_NUMS.items():
    
    for i in range(num):
        tTAC = all_tacs['noise-free'][a_model][f"set-{i:02}"]
        gen_patlak_figure(pTAC=pTAC, tTAC=tTAC, t_thresh_in_mins=35.0)
        
        plt.show()
#     break

In [25]:
for a_model, num in MODELS_NUMS.items():
    myFig, myAx = plt.subplots(1, 2, constrained_layout=True, figsize=[8, 4], dpi=120, linewidth=3.0, edgecolor='k')
    for i in range(num):
        _params = model_params[a_model][f"set-{i:02}"]
        _tTAC = all_tacs['noise-free'][a_model][f"set-{i:02}"]
        patlak_vars = gen_patlak_plot(pTAC, _tTAC)
        
        myAx[0].plot(*patlak_vars, '.-', alpha=0.95, lw=2, 
                     #label=params_dict_to_label(_params['params']), 
                     color=f"C{i}",
                     ms=8, mec='k', mew=0.5)
        myAx[1].plot(*patlak_vars, '.-', alpha=0.95, lw=2, color=f"C{i}", ms=8, mec='k', mew=0.5)
    
    myAx[0].set(title=f"Linear Plot")
    myAx[1].set(title=f"Log-Log Plot", xscale='log', yscale='log')
    myAx[0].set_xlabel(patlak_xLab, fontsize=20)
    myAx[0].set_ylabel(patlak_yLab, fontsize=20)
    myAx[1].set_xlabel(patlak_xLab, fontsize=20)
    myAx[1].set_ylabel(patlak_yLab, fontsize=20)
    myFig.suptitle(f"{_params['model']} Patlak Plots", fontweight='bold')
#     myFig.legend(bbox_to_anchor=(1.0, 0.6), loc='center left', title='Parameters')
    plt.show()

## Logan Plots

In [ ]:
def gen_logan_plot(pTAC, tTAC):
    x = pet_gph.cumulative_trapezoidal_integral(xdata=pTAC[0], ydata=pTAC[1]) / tTAC[1]
    y = pet_gph.cumulative_trapezoidal_integral(xdata=tTAC[0], ydata=tTAC[1]) / tTAC[1]
    return np.array([x, y])

def gen_alt_logan_plot(pTAC, tTAC):
    x = pet_gph.cumulative_trapezoidal_integral(xdata=pTAC[0], ydata=pTAC[1]) / pTAC[1]
    y = pet_gph.cumulative_trapezoidal_integral(xdata=tTAC[0], ydata=tTAC[1]) / pTAC[1]
    return np.array([x, y])
    

# Testing Implementation From Package

In [327]:
import petpal.visualizations.graphical_plots as pet_plt
reload(pet_plt)

## Patlak Plots

In [338]:
for a_model, num in MODELS_NUMS.items():
    
    for i in range(num):
        tTAC = all_tacs['noise-free'][a_model][f"set-{i:02}"]
        patlak_plot = pet_plt.PatlakPlot(tTAC=tTAC, pTAC=pTAC, t_thresh_in_mins=45.0)
        patlak_plot.generate_figure(line_kwargs=dict(lw=2, alpha=0.95, color='red', label=patlak_plot.generate_label_from_fit_params()), 
                                    shading_kwargs=dict(color='palegreen', alpha=0.2),
                                    data_kwargs=dict(alpha=0.85, color='k', marker='.'))
        plt.show()
#     break

## Logan Plots

In [339]:
for a_model, num in MODELS_NUMS.items():
    
    for i in range(num):
        tTAC = all_tacs['noise-free'][a_model][f"set-{i:02}"]
        patlak_plot = pet_plt.LoganPlot(tTAC=tTAC, pTAC=pTAC, t_thresh_in_mins=45.0)
        patlak_plot.generate_figure()
        
        
        plt.show()
#     break

## Alt Logan Plots

In [324]:
for a_model, num in MODELS_NUMS.items():
    
    for i in range(num):
        tTAC = all_tacs['noise-free'][a_model][f"set-{i:02}"]
        patlak_plot = pet_plt.AltLoganPlot(tTAC=tTAC, pTAC=pTAC, t_thresh_in_mins=45.0)
        patlak_plot.generate_figure()
        
        plt.show()
#     break

# Generating Plots From Reading JSON Files Of Analyses

## Testing Reading JSON From Analysis

In [347]:
test_json = "/Users/furqandar/Desktop/Work/BrierLab/PracticeData/4DPET/test-patlak-analysis-props.json"

test_params = {}

with open(test_json) as dFile:
    test_params = json.load(fp=dFile)
    
test_params

In [348]:
ptac_path = test_params["FilePathPTAC"]
ttac_path = test_params["FilePathTTAC"]
t_thresh_time = test_params["ThresholdTime"]



In [350]:
logan_plot = pet_plt.PatlakPlot(pTAC=pet_grph._safe_load_tac(ptac_path),
                  tTAC=pet_grph._safe_load_tac(ttac_path),
                  t_thresh_in_mins=t_thresh_time)
logan_plot.generate_figure()

plt.show()

# Testing File Path Based Image Saving

In [2]:
import petpal.visualizations.graphical_plots as pet_plt
from importlib import reload

In [3]:
reload(pet_plt)

<module 'petpal.visualizations.graphical_plots' from '/Users/furqandar/Library/Python/3.9/lib/python/site-packages/petpal/visualizations/graphical_plots.py'>

In [4]:
import json
test_json = "/Users/furqandar/Desktop/Work/BrierLab/PracticeData/4DPET/test-patlak-analysis-props.json"

test_params = {}

with open(test_json) as dFile:
    test_params = json.load(fp=dFile)
    
test_params

{'FilePathPTAC': '/Users/furqandar/Desktop/Work/BrierLab/Code/PETProcessing/data/tcm_tacs/fdg_plasma_clamp_evenly_resampled.txt',
 'FilePathTTAC': '/Users/furqandar/Desktop/Work/BrierLab/Code/PETProcessing/data/tcm_tacs/serial_2tcm_k4zero/gaussian_noise/tac_2tcm_k4zero_set-01.txt',
 'MethodName': 'patlak',
 'ThresholdTime': 30.0,
 'StartFrameTime': 30.1196,
 'EndFrameTime': 119.876,
 'NumberOfPointsFit': 299,
 'Slope': 0.16389227863069047,
 'Intercept': 0.75737641363798,
 'RSquared': 0.9802031815646701}

In [5]:
ptac_path = test_params["FilePathPTAC"]
ttac_path = test_params["FilePathTTAC"]
t_thresh_time = test_params["ThresholdTime"]



In [6]:
grph_plot = pet_plt.Plot(input_tac_path=ptac_path,
                         roi_tac_path=ttac_path,
                         threshold_in_mins=30.0,
                         method_name='patlak', 
                         output_directory='../../../../PracticeData/4DPET/',
                         output_filename_prefix="plot")
grph_plot.save_figure()